# Topic analiza

## Vzpostavitev Pythonovega okolja

In [1]:
# If needed, run shell commands in R via system()
system("which python")
system("python --version")
system("sudo apt-get update -y")
system("sudo apt-get install -y python3-dev")
system("sudo apt-get install -y python3-venv")


## Namestitev vmesnika za Python

In [2]:
install.packages("reticulate")
library(reticulate)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘RcppTOML’, ‘here’, ‘png’




## Ustvarjanje Pythonovega virtualnega okolja

In [3]:
# virtualenv_create("bertopic", python = "/usr/local/bin/python")
system("python3 -m venv /root/.virtualenvs/bertopic")
system("pip install virtualenv")
system("virtualenv /root/.virtualenvs/bertopic")


# Namestitev Pythonovih knjižnic

In [4]:
library(reticulate)
use_virtualenv("bertopic", required = TRUE)
py_install("pandas", envname = "bertopic")
py_install(c("numpy", "matplotlib"), envname = "bertopic")
py_install("bertopic", envname = "bertopic")
py_install("umap-learn", envname = "bertopic")
py_install("hdbscan", envname = "bertopic")
py_install("scikit-learn", envname = "bertopic")
py_install("plotly", envname = "bertopic")

Using virtual environment 'bertopic' ...


+ /root/.virtualenvs/bertopic/bin/python -m pip install --upgrade --no-user pandas



Using virtual environment 'bertopic' ...


+ /root/.virtualenvs/bertopic/bin/python -m pip install --upgrade --no-user numpy matplotlib



Using virtual environment 'bertopic' ...


...

+ /root/.virtualenvs/bertopic/bin/python -m pip install --upgrade --no-user hdbscan



Using virtual environment 'bertopic' ...


+ /root/.virtualenvs/bertopic/bin/python -m pip install --upgrade --no-user scikit-learn



Using virtual environment 'bertopic' ...


+ /root/.virtualenvs/bertopic/bin/python -m pip install --upgrade --no-user plotly



## Priklic Pythonovih knjižnic

In [5]:
# Import necessary Python modules
np <- import("numpy")
umap <- import("umap")
UMAP <- umap$UMAP
hdbscan <- import("hdbscan")
HDBSCAN <- hdbscan$HDBSCAN
sklearn <- import("sklearn")
CountVectorizer <- sklearn$feature_extraction$text$CountVectorizer
bertopic <- import("bertopic")
plotly <- import("plotly")
datetime <- import("datetime")


## Namestitev knjižnic R

In [6]:
install.packages(c("dplyr", "tidyr", "tictoc", "htmltools", "htmlwidgets"))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [7]:
install.packages("devtools")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [8]:
devtools::install_github("tpetric7/bertopicr")


── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpVbLflT/remotes276144083e8/tpetric7-bertopicr-fee33d5/DESCRIPTION’ ... OK
* preparing ‘bertopicr’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘bertopicr_0.3.0.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



## Priklic knjižnic R

In [10]:
library(dplyr)
library(tidyr)
# library(quanteda)
library(tictoc)
library(htmltools)
library(htmlwidgets)
# library(arrow)
library(bertopicr)

## Branje podatkovnega niza

In [11]:
input_folder <- "/content/"
input_file <- "ucb_uni_cleaned.rds"
dataset <- readRDS(file.path(input_folder, input_file))
names(dataset)
dim(dataset)

[1] "Doc_ID"          "School"          "veda"            "program"        
 [5] "book_number"     "year"            "Text"            "sentence_id"    
 [9] "sent_length"     "seq_id"          "text_clean"      "sentence_length"

[1] 76642    12

## Branje nezaželenih besed

In [12]:
input_folder <- "/content/"
input_file <- "all_stopwords.txt"
all_stopwords <- read_lines(file.path(input_folder, input_file))

## Izbor besedilnega stolpca

In [13]:
texts_cleaned = dataset$text_clean
# titles = dataset$doc_id
titles = dataset$Doc_ID
# timestamps <- as.list(dataset$date)
timestamps <- as.integer(dataset$year)

texts_cleaned[[1]]

[1] "univerza v ljubljani biotehniška fakulteta oddelek za agronomijo prof dr katja vadna čl icz o univerza v ljubljani biotehniška fakulteta cip kataložni zapis o publikaciji narodna in univerzitetna knjižnica ljubljana vadnal katja agrarna ekonomika osnove trg država katja vadnal l izd ljubljana biotehniška fakulteta oddelek za agronomijo isbn recenzija prof dr jernej turk kmetijska fakulteta univerza v mariboru doc dr andrej udovč biotehniška fakulteta univerza v ljubljani lektoriranje prof alenka raič blažič oblikovanje franci"

## Vektorska reprezentacija (embeddings)

In [14]:
# Embed the sentences
py <- import_builtins()
sentence_transformers <- import("sentence_transformers")
SentenceTransformer <- sentence_transformers$SentenceTransformer
# embedding_model = SentenceTransformer("BAAI/bge-m3")
embedding_model = SentenceTransformer("cjvt/sloberta-trendi-topics")
embeddings = embedding_model$encode(texts_cleaned, show_progress_bar=TRUE)


## Redukcija dimenzij in grozdenje

In [15]:
# Initialize UMAP and HDBSCAN models
umap_model <- UMAP(n_neighbors=15L, n_components=5L, min_dist=0.0, metric='cosine', random_state=42L)


In [16]:
hdbscan_model <- HDBSCAN(min_cluster_size=50L, min_samples = 20L, metric='euclidean', cluster_selection_method='eom', gen_min_span_tree=TRUE, prediction_data=TRUE)


In [17]:
# Initialize CountVectorizer
vectorizer_model <- CountVectorizer(min_df=2L, ngram_range=tuple(1L, 3L),
                                    max_features = 10000L, max_df = 50L,
                                    stop_words = all_stopwords)
sentence_vectors <- vectorizer_model$fit_transform(texts_cleaned)
sentence_vectors_dense <- np$array(sentence_vectors)
sentence_vectors_dense <- py_to_r(sentence_vectors_dense)


## Reprezentacijski modeli

In [18]:
# Initialize representation models
keybert_model <- bertopic$representation$KeyBERTInspired()
# openai <- import("openai")
# OpenAI <- openai$OpenAI
# ollama <- import("ollama")

# # Point to the local server (ollama or lm-studio)
# client <- OpenAI(base_url = 'http://localhost:11434/v1', api_key='ollama')
# # client <- OpenAI(base_url = 'http://localhost:1234/v1', api_key='lm-studio')

# prompt <- "
# I have a topic that contains the following documents:
# [DOCUMENTS]
# The topic is described by the following keywords: [KEYWORDS]

# Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
# topic: <topic label>
# "

# # download an appropriate local LLM for your computer with ollama/lm-studio
# openai_model <- bertopic$representation$OpenAI(client,
#                                                model = "llama3.1:8b-instruct-fp16",
#                                                exponential_backoff = TRUE,
#                                                chat = TRUE,
#                                                prompt = prompt)

# # downlaod a spacy language model from spacy.io before use here
# pos_model <- bertopic$representation$PartOfSpeech("sl_core_news_trf")

# diversity set relatively high to reduce repetition of keyword word forms
mmr_model <- bertopic$representation$MaximalMarginalRelevance(diversity = 0.5)

# Combine all representation models
representation_model <- list(
  "KeyBERT" = keybert_model,
  # "OpenAI" = openai_model,
  # "POS" = pos_model,
  "MMR" = mmr_model
)


## Vzpostavitev modela BERTopic

In [19]:
# Initialize BERTopic model with pipeline models and hyperparameters
BERTopic <- bertopic$BERTopic
topic_model <- BERTopic(
  embedding_model = embedding_model,
  umap_model = umap_model,
  hdbscan_model = hdbscan_model,
  vectorizer_model = vectorizer_model,
  # zeroshot_topic_list = zeroshot_topic_list,
  # zeroshot_min_similarity = 0.6, # 0.85
  representation_model = representation_model,
  calculate_probabilities = TRUE,
  top_n_words = 10L,
  verbose = TRUE
)


## Učenje modela BERTopic

In [20]:
tictoc::tic()

# Fit the model and transform the texts
fit_transform <- topic_model$fit_transform(texts_cleaned, embeddings)
topics <- fit_transform[[1]]
# probs <- fit_transform[[2]]

# Now transform the texts to get the updated probabilities
transform_result <- topic_model$transform(texts_cleaned)
probs <- transform_result[[2]]  # Extract the updated probabilities

tictoc::toc()


871.508 sec elapsed


## Časovna dimenzija

In [21]:
# Converting R Date to Python datetime
datetime <- import("datetime")

# timestamps <- as.list(dataset$date)
timestamps <- as.integer(dataset$year)

# # Convert each R date object to an ISO 8601 string
# timestamps <- lapply(timestamps, function(x) {
#   format(x, "%Y-%m-%dT%H:%M:%S")  # ISO 8601 format
# })

# Dynamic topic model
topics_over_time  <- topic_model$topics_over_time(texts_cleaned, timestamps, nr_bins=20L, global_tuning=TRUE, evolution_tuning=TRUE)


## Rezultati topic analize

In [22]:
# Combine results with additional columns
results <- dataset |>
  mutate(Topic = topics,
         Probability = apply(probs, 1, max))  # Assuming the highest probability for each sentence

results <- results |>
  mutate(row_id = row_number()) |>
  select(row_id, everything())

head(results,10)

row_id,Doc_ID,School,veda,program,book_number,year,Text,sentence_id,sent_length,seq_id,text_clean,sentence_length,Topic,Probability
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<int>,<int>,<dbl>
1,UL_AGRARNA EKONOMIKA_B,UL,B,AGRARNA EKONOMIKA,(1),2003,"univerza v ljubljani biotehniška fakulteta oddelek za agronomijo prof , dr . katja vadna čl\ 'icz^o- univerza v ljubljani biotehniška fakulteta cip-kataložni zapis o publikaciji narodna in univerzitetna knjižnica ljubljana 338.43.01 ( 075.8 ) vadnal , katja agrarna ekonomika : osnove , trg , država / katja vadnal . - l izd . - ljubljana : biotehniška fakulteta , oddelek za agronomijo , 2003 isbn 961-6275-10-0 122001664 recenzija : prof. dr. jernej turk , kmetijska fakulteta , univerza v mariboru doc . dr. andrej udovč , biotehniška fakulteta , univerza v ljubljani lektoriranje : prof. alenka raič-blažič oblikovanje : franci",1,86,1,univerza v ljubljani biotehniška fakulteta oddelek za agronomijo prof dr katja vadna čl icz o univerza v ljubljani biotehniška fakulteta cip kataložni zapis o publikaciji narodna in univerzitetna knjižnica ljubljana vadnal katja agrarna ekonomika osnove trg država katja vadnal l izd ljubljana biotehniška fakulteta oddelek za agronomijo isbn recenzija prof dr jernej turk kmetijska fakulteta univerza v mariboru doc dr andrej udovč biotehniška fakulteta univerza v ljubljani lektoriranje prof alenka raič blažič oblikovanje franci,74,0,0.9999999
2,UL_AGRARNA EKONOMIKA_B,UL,B,AGRARNA EKONOMIKA,(1),2003,"kramarič tisk : forma 1 , vrhpolje 56 , 1251 mora '' x ~ v skladu s sklepom senata oddelka za agronomijo , z dne 19.2.2003 , se univerzitetni učbenik agrarna ekonomika . osnove , trg , država , katerega avtorica je prof. dr. katja vadnal , na biotehniški fakulteti uporablja kot predpisano učno gradivo . založba : biotehniška fakulteta univerze \ i naklada : 500 izvodov dekan biotehniške fakultete prof. dr. jože resnik , 1. r. univerza v ljubljani biotehniška fakulteta oddelek za agronomijo katja vadnal agrarna ekonomika osnove , trg , država ljubljana , 2003 vsebina predgovor 9",2,83,2,kramarič tisk forma vrhpolje mora x v skladu s sklepom senata oddelka za agronomijo z dne se univerzitetni učbenik agrarna ekonomika osnove trg država katerega avtorica je prof dr katja vadnal na biotehniški fakulteti uporablja kot predpisano učno gradivo založba biotehniška fakulteta univerze i naklada izvodov dekan biotehniške fakultete prof dr jože resnik r univerza v ljubljani biotehniška fakulteta oddelek za agronomijo katja vadnal agrarna ekonomika osnove trg država ljubljana vsebina predgovor,72,0,0.9999810
3,UL_AGRARNA EKONOMIKA_B,UL,B,AGRARNA EKONOMIKA,(1),2003,"prvi del : razvoj in metoda agrarne ekonomike 11 1.1 razvoj agrarne ekonomike kot znanstvene 12 discipline 1.1.1 poreklo imena 12 1.1.2 razvoj agrarne ekonomike 13 1.2 definicija agrarne ekonomike 16 1.3 agrarnoekonomsko raziskovanje 17 drugi del : osnove agrarne ekonomike 23 2.1 kmetijstvo in podeželje 24 2.1.1 kmetijstvo , kajje to ? 24 2.1.2 podeželje 29 2.2 prvine kmetijskega pridelovanja 31 2.2.1 naravni viri 31 2.2.2 delo 32 2.2.3 izdelane prvine pridelovanja 35 2.2.4 proizvodna funkcija 37 2.2.5 biotehiški napredek 40 2.3 zakon redkosti 41 2.3.1 redkost prvin pridelovanja in pridelkov 41 2.3.2 krivulja različnih proizvodnih možnosti 42 2.4",3,125,3,prvi del razvoj in metoda agrarne ekonomike razvoj agrarne ekonomike kot znanstvene discipline poreklo imena razvoj agrarne ekonomike definicija agrarne ekonomike agrarnoekonomsko raziskovanje drugi del osnove agrarne ekonomike kmetijstvo in podeželje kmetijstvo kajje to podeželje prvine kmetijskega pridelovanja naravni viri delo izdelane prvine pridelovanja proizvodna funkcija biotehiški napredek zakon redkosti redkost prvin pridelovanja in pridelkov krivulja različnih proizvodnih možnosti,59,0,1.0000000
4,UL_AGRARNA EKONOMIKA_B,UL,B,AGRARNA EKONOMIKA,(1),2003,vrste kmetijskih gospodarstev 46 2.4.1 kmetijsko gospodarstvo 46 2.4.2 k

## Informacije o dokumentih

In [23]:
library(bertopicr)
document_info_df <- get_document_info_df(model = topic_model,
                                         texts = texts_cleaned,
                                         drop_expanded_columns = TRUE)
document_info_df |> head()

Document,Topic,Name,Top_n_words,Probability,Representative_document,Representation,KeyBERT,MMR,Representative_Docs
<chr>,<dbl>,<chr>,<chr>,<dbl>,<lgl>,<chr>,<chr>,<chr>,<chr>
univerza v ljubljani biotehniška fakulteta oddelek za agronomijo prof dr katja vadna čl icz o univerza v ljubljani biotehniška fakulteta cip kataložni zapis o publikaciji narodna in univerzitetna knjižnica ljubljana vadnal katja agrarna ekonomika osnove trg država katja vadnal l izd ljubljana biotehniška fakulteta oddelek za agronomijo isbn recenzija prof dr jernej turk kmetijska fakulteta univerza v mariboru doc dr andrej udovč biotehniška fakulteta univerza v ljubljani lektoriranje prof alenka raič blažič oblikovanje franci,0,0_za_ki_se_pri,za - ki - se - pri - ali - lahko - pa - tudi - kot - po,0.7925759,FALSE,"za, ki, se, pri, ali, lahko, pa, tudi, kot, po","različnih, ki ga, ki jih, ni, zelo, med, ko, drugi, primer, ima","za, ali, lahko, od, med, ni, ko, bo, primer, oziroma","vročinske oblive in vpliv na vaginalno sluznico med kasnimi neželenimi učinki je pomembno povečanje tveganja za rak maternice zaradi delovanja na endometrij večje je tudi tveganje za venske tromboze in razvoj katarakte zdravila iz vrste aromataznih zaviralcev anastrozol letrozol eksemestan nimajo kasnih neželenih učinkov na maternično sluznico tudi tveganje za tromboze je manjše kot pri tamoksifenu pogosto pa povzročajo mišično sklepne bolečine in občutek jutranje okorelosti kar je včasih pri nekaterih bolnicah razlog da opustijo jemanje zdravila sicer aromatazni zaviralci zlasti pri že prisotni osteopeniji povečajo tveganje za osteoporozo in zlome kosti povzročajo tudi prehodno hiperlipidemijo njihov vpliv na kardiovaskularni sistem, pri ponovljenem merjenju ali pa vektorizacija matrike in koeficient korelacije kot mera zanesljivosti po metodi retesta ocenjevanje kakovosti merjenja velikosti lokalnega omrežja ali mer centralnosti odstotek identičnih oseb ali koeficient korelacije med merami centralnosti kot meri stabilnosti po metodi retesta v nadaljevanju je najprej predstavljeno ocenjevanje zanesljivosti merjenja na vektorizirani relacijskimatriki vektorizacija matrike je postopek pri katerem iz matrike dobimo stolpec podatkov ki ga lahko obravnavamo kotstandardno spremenljivko pri vektorizaciji izmatrike naredimo stolpec tako da posamezne stolpce zapisujemo enega za drugim omrežje je tako spremenljivka posamezna vrednostmatrike, vrvni bobni vrvni bobni mostnih žerjavov omogočajo dvig bremen površina vrvnega bobna je lahko gladka če se vrv navija na boben v več plasteh pogosteje se v praksi uporablja ožlebljen vrvni boben kjer se vrv navija v eni plasti tako je vrv v žlebu bobna vodena in ne drsi po vrvnem bobnu vrvni bobni so lahko liti iz sive litine večji vrvni bobni pa so varjeni iz jeklene pločevine mostni žerjavi v intralogistiki dimenzioniranje vrvnega bobna po standardu en tudi dimenzioniranje vrvnega bobna izvedemo na"
kramarič tisk forma vrhpolje mora x v skladu s sklepom senata oddelka za agronomijo z dne se univerzitetni učbenik agrarna ekonomika osnove trg država katerega avtorica je prof dr katja vadnal na biotehniški fakulteti uporablja kot predpisano učno gradivo založba biotehniška fakulteta univerze i naklada izvodov dekan biotehniške fakultete prof dr jože resnik r univerza v ljubljani biotehniška fakulteta oddelek za agronomijo katja vadnal agrarna ekonomika osnove trg država ljubljana vsebina predgovor,0,0_za_ki_se_pri,za - ki - se - pri - ali - lahko - pa - tudi - kot - po,0.9435439,FALSE,"za, ki, se, pri, ali, lahko, pa, tudi, kot, po","različnih, ki ga, ki jih, ni, zelo, med, ko, drugi, primer, ima","za, ali, lahko, od, med, ni, ko, bo, primer, oziroma","vročinske oblive in vpliv na vaginalno sluznico med kasnimi neželenimi učinki je pomembno povečanje tveganja za rak maternice zaradi delovanja na endometrij večje je tudi tveganje za venske tromboze in razvoj katarakte zdravila iz vrste aromataznih zaviralcev anastrozol letrozol eksemestan nimajo kasni

In [24]:
# Create a data frame similar to df_docs
df_docs <- tibble(Topic = results$Topic,
                  Document = results$text_clean,
                  probs = results$Probability)
rep_docs <- get_most_representative_docs(df = df_docs,
                                         topic_nr = 3,
                                         n_docs = 5)
unique(rep_docs)

[1] "the financing of terrorism sprejeta z resolucijo generalne skupščine združenih narodov št in odprta za podpis decembra v veljavi od aprila število pogodbenic april daytonski sporazum dayton agreement dogovorjen v daytonu s strani predstavnikov republike bosne in hercegovine republike hrvaške in zvezne republike jugoslavije decembra in podpisan v parizu decembra vstopil v veljavo s podpisom dokument zapišemo v besedilo z naslovom dokumenta in večinoma z letnico njegovega nastanka če je dokument ki"                                                                                                                                                  
[2] "b imata spor glede meje dogovorita se za posredovanje predsednika države c ki naj bi pripeljalo do rešitve najkasneje do konca leta sicer pa bo treba poiskati drug način reševanja spora posredovanje je bilo neuspešno in je bil sestanek na katerem sta bila prisotna zunanja ministra držav a in b ter posebni odposlanec predsednika države c dogovorjeno je bilo da se s posredovanjem vseeno nadaljuje tako da se drugimi načini reševanja spora počaka še do konca leta zapisnik s sestanka so podpisali vsi trije v nasprotju z"                                                                                                                               
[3] "i poglavje cilji in načela člen cilji združenih narodov so varovati mednarodni mir in varnost ter v ta namen izvajati učinkovite kolektivne ukrepe da se preprečijo in odvrnejo grožnje miru in da se zatrejo agresivna dejanja ali druge kršitve miru in si prizadevati da se mednarodni spori ali pa situacije ki bi utegnile privesti do kršitve miru zgladijo ali uredijo z mirnimi sredstvi in v skladu z načeli pravičnosti in mednarodnega prava razvijati med narodi prijateljske odnose sloneče na spoštovanju načela enakopravnosti ljudstev in njihove samoodločbe ter izvajati"                                                                             
[4] "države ali pa ki bi bila kako drugače nezdružljiva s cilji združenih narodov vsi člani naj v polni meri pomagajo združenim narodom v sleherni akciji ki bi jo ti izvajali v skladu s to ustanovno listino in naj se vzdržujejo dajanja pomoči katerikoli državi proti kateri bi združeni narodi izvajali preventivno ali prisilno akcijo organizacija naj zagotovi da bodo države ki niso članice združenih narodov delale v skladu s temi načeli kolikor bi to bilo potrebno za ohranitev mednarodnega miru in varnosti nobena določba te ustanovne listine ne daje"                                                                                                   
[5] "in jo ratificirajo v skladu s členom člen lahko postanejo članice združenih narodov vse druge miroljubne države ki prevzamejo obveznosti vsebovane v tej ustanovni listini in ki so po presoji organizacije sposobne in voljne spolnjevati te obveznosti vsaka takšna država se sprejme za članico združenih narodov z odločbo generalne skupščine na priporočilo varnostnega sveta člen članu združenih narodov zoper katerega je varnostni svet ukrenil preventivno ali prisilno akcijo sme generalna skupščina na priporočilo varnostnega sveta ustaviti uživanje članskih pravic in privilegijev uživanje teh pravic in privilegijev sme vzpostaviti varnostni svet"

## Informacije o temah (topics)

In [25]:
topic_info_df <- get_topic_info_df(model = topic_model,
                                   drop_expanded_columns = TRUE)
head(topic_info_df)

Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
-1,33,-1_matematik_const robtarget target__robtarget target__target_ const robtarget,"matematik, const robtarget target_, robtarget target_, target_ const robtarget, robtarget target_ const, target_ const, const robtarget, robtarget, target_, const","target_ const robtarget, target_ const, robtarget target_ const, const robtarget target_, robtarget target_, target_, verjetnostni, analitične, eliptičnih, geometrija","const robtarget target_, robtarget target_, robtarget target_ const, francoski, verjetnostni račun, teorija števil, verjetnostni, astronom, algebra, švicarski matematik","e const robtarget target_ e const robtarget target_ e const robtarget target_ e const robtarget target_, const robtarget target_ e const robtarget target_ e const robtarget target_ e const robtarget target_, e const robtarget target_ e const robtarget target_ e const robtarget target_ e const robtarget target_"
0,76238,0_za_ki_se_pri,"za, ki, se, pri, ali, lahko, pa, tudi, kot, po","različnih, ki ga, ki jih, ni, zelo, med, ko, drugi, primer, ima","za, ali, lahko, od, med, ni, ko, bo, primer, oziroma","vročinske oblive in vpliv na vaginalno sluznico med kasnimi neželenimi učinki je pomembno povečanje tveganja za rak maternice zaradi delovanja na endometrij večje je tudi tveganje za venske tromboze in razvoj katarakte zdravila iz vrste aromataznih zaviralcev anastrozol letrozol eksemestan nimajo kasnih neželenih učinkov na maternično sluznico tudi tveganje za tromboze je manjše kot pri tamoksifenu pogosto pa povzročajo mišično sklepne bolečine in občutek jutranje okorelosti kar je včasih pri nekaterih bolnicah razlog da opustijo jemanje zdravila sicer aromatazni zaviralci zlasti pri že prisotni osteopeniji povečajo tveganje za osteoporozo in zlome kosti povzročajo tudi prehodno hiperlipidemijo njihov vpliv na kardiovaskularni sistem, pri ponovljenem merjenju ali pa vektorizacija matrike in koeficient korelacije kot mera zanesljivosti po metodi retesta ocenjevanje kakovosti merjenja velikosti lokalnega omrežja ali mer centralnosti odstotek identičnih oseb ali koeficient korelacije med merami centralnosti kot meri stabilnosti po metodi retesta v nadaljevanju je najprej predstavljeno ocenjevanje zanesljivosti merjenja na vektorizirani relacijskimatriki vektorizacija matrike je postopek pri katerem iz matrike dobimo stolpec podatkov ki ga lahko obravnavamo kotstandardno spremenljivko pri vektorizaciji izmatrike naredimo stolpec tako da posamezne stolpce zapisujemo enega za drugim omrežje je tako spremenljivka posamezna vrednostmatrike, vrvni bobni vrvni bobni mostnih žerjavov omogočajo dvig bremen površina vrvnega bobna je lahko gladka če se vrv navija na boben v več plasteh pogosteje se v praksi uporablja ožlebljen vrvni boben kjer se vrv navija v eni plasti tako je vrv v žlebu bobna vodena in ne drsi po vrvnem bobnu vrvni bobni so lahko liti iz sive litine večji vrvni bobni pa so varjeni iz jeklene pločevine mostni žerjavi v intralogistiki dimenzioniranje vrvnega bobna po standardu en tudi dimenzioniranje vrvnega bobna izvedemo na"
1,194,1_dediščine_dediščino_varstva_za,"dediščine, dediščino, varstva, za, ki, kulturne, kulturne dediščine, ali, tudi, del","spomenikov, konservatorskega načrta, arheološka, arheoloških, kulturne dediščine, arheološke, varstva dediščine, najdišča, kulturne, arheološke raziskave","dediščino, kulturne, kulturne dediščine, zvkd, arheološke, dediščina, ohranjanje, kulturno, ohranjanja, kulturno dediščino","enoto dediščine iz registra v kratkem bo sistem evrd nadgrajen v sistem varstvenih območij dediščine kot ga predpisuje člen zvkd na podlagi meril kot so skupen zgodovinski kontekst nepremične dediščine sorodne morfološke značilnosti in vrednote dediščine v prostoru in topografske enovitosti bodo s podzakonskimi akti določena varstvena območja in pripadajoče varstvene usmeritve na podlagi podzakonskih aktov o varstvenih o

In [26]:
topics_df <- get_topics_df(model = topic_model)
head(topics_df, 10)

Word,Score,Topic
<chr>,<dbl>,<dbl>
matematik,0.2347178,-1
const robtarget target_,0.1890793,-1
robtarget target_,0.1890793,-1
target_ const robtarget,0.1861988,-1
robtarget target_ const,0.1861988,-1
target_ const,0.1861988,-1
const robtarget,0.1798023,-1
robtarget,0.1747537,-1
target_,0.1740939,-1


## Interaktivni stolpčni diagram

In [27]:
visualize_barchart(model = topic_model,
                   filename = "topics_topwords_interactive_barchart.html", # default
                   open_file = FALSE) # TRUE enables output in browser

## Iskanje teme (topics)

In [28]:
find_topics_df(model = topic_model,
               queries = "podnebne spremembe", # user input
               top_n = 10, # default
               return_tibble = TRUE) # default

keyword,topics,similarity
<chr>,<int>,<dbl>
podnebne spremembe,1,0.4470647
podnebne spremembe,3,0.3507194
podnebne spremembe,0,0.3044683
podnebne spremembe,-1,0.3020374
podnebne spremembe,2,-0.1023505


In [29]:
find_topics_df(model = topic_model,
                               queries = c("podnebne spremembe", "trajnostni razvoj"),
                               top_n = 5)

keyword,topics,similarity
<chr>,<int>,<dbl>
podnebne spremembe,1,0.44706467
podnebne spremembe,3,0.35071943
podnebne spremembe,0,0.30446834
podnebne spremembe,-1,0.30203738
podnebne spremembe,2,-0.10235054
trajnostni razvoj,1,0.47454733
trajnostni razvoj,3,0.40161397
trajnostni razvoj,-1,0.37443237
trajnostni razvoj,0,0.34853539


## Pridobivanje ključnih besed za temo (topic)

In [30]:
get_topic_df(model = topic_model,
                           topic_number = 0,
                           top_n = 5, # default is 10
                           return_tibble = TRUE) # default

Word,Score,Topic
<chr>,<dbl>,<int>
za,0.05889539,0
ki,0.05256292,0
se,0.05058601,0
pri,0.04001787,0
ali,0.03601871,0


## Interaktivna vizualizacija distribucije tem (topics)

In [31]:
# default filename: topic_dist_interactive.html
visualize_distribution(model = topic_model,
                       text_id = 1, # user input
                       probabilities = probs) # see model training

## Interaktivna vizualizacija razdalje med temami (topics)

In [32]:
visualize_topics(model = topic_model,
                 filename = "intertopic_distance_map") # default name

## Podobnost tem (topics)

In [33]:
visualize_heatmap(model = topic_model,
                  filename = "topics_similarity_heatmap",
                  auto_open = FALSE)

## Hierarhija tem (topics)

In [34]:
visualize_hierarchy(model = topic_model,
                    hierarchical_topics = NULL, # default
                    filename = "topic_hierarchy", # default name, html extension
                    auto_open = FALSE) # TRUE enables output in browser

In [35]:
hierarchical_topics = topic_model$hierarchical_topics(texts_cleaned)
visualize_hierarchy(model = topic_model,
                    hierarchical_topics = hierarchical_topics,
                    filename = "topic_hierarchy", # default name, html extension
                    auto_open = FALSE) # TRUE enables output in browser

## Interaktivna vizualazacija dokumentov 2D in 3D

In [36]:
# Reduce dimensionality of embeddings using UMAP
reduced_embeddings <- umap$UMAP(n_neighbors = 10L, n_components = 2L, min_dist = 0.0, metric = 'cosine')$fit_transform(embeddings)

visualize_documents(model = topic_model,
                    texts = texts_cleaned,
                    reduced_embeddings = reduced_embeddings,
                    filename = "visualize_documents", # default extension html
                    auto_open = FALSE) # TRUE enables output in browser

In [37]:
# Reduce dimensionality of embeddings using UMAP
reduced_embeddings <- umap$UMAP(n_neighbors = 10L, n_components = 3L, min_dist = 0.0, metric = 'cosine')$fit_transform(embeddings)

visualize_documents_3d(model = topic_model,
                       texts = texts_cleaned,
                       reduced_embeddings = reduced_embeddings,
                       custom_labels = FALSE, # default
                       hide_annotation = TRUE, # default
                       tooltips = c("Topic", "Name", "Probability", "Text"), # deault
                       filename = "visualize_documents_3d", # default name
                       auto_open = FALSE) # TRUE enables output in browser

## Časovna razporeditev tem (topics)

In [38]:
visualize_topics_over_time(model = topic_model,
                           # see Topic Dynamics section above
                           topics_over_time_model = topics_over_time,
                           top_n_topics = 10, # default is 20
                           filename = "topics_over_time") # default, html extension

## Razporeditev tem (topics) po skupinah

In [39]:
classes = as.list(dataset$veda) # text types
topics_per_class = topic_model$topics_per_class(texts_cleaned, classes=classes)

visualize_topics_per_class(model = topic_model,
                           topics_per_class = topics_per_class,
                           start = 0, # default
                           end = 10, # default
                           filename = "topics_per_class", # default, html extension
                           auto_open = FALSE) # TRUE enables output in browser

ERROR: Error in value[[3L]](cond): Error generating the visualization: IndexError: index 0 is out of bounds for axis 0 with size 0
Run `reticulate::py_last_error()` for details.


## Shrani / download interaktivne vizualizacije

In [40]:
# prompt: download all html files in the /content/ folder to the local computer

# This code requires the "utils" package which is a base R package
# so you shouldn't need to install it

# List all HTML files in the /content directory
html_files <- list.files(path = "/content/", pattern = "\\.html$", full.names = TRUE)

print(html_files)

[1] "/content//intertopic_distance_map.html"             
[2] "/content//topic_dist_interactive.html"              
[3] "/content//topic_hierarchy.html"                     
[4] "/content//topics_over_time.html"                    
[5] "/content//topics_similarity_heatmap.html"           
[6] "/content//topics_topwords_interactive_barchart.html"
[7] "/content//visualize_documents_3d.html"              
[8] "/content//visualize_documents.html"                 


In [41]:
# Check if there are any HTML files
if (length(html_files) > 0) {
  # Download each HTML file
  for (file in html_files) {
    download.file(file, destfile = file)
    # Print a message to indicate the download is complete
    cat(paste("Downloaded:", file, "to", local_file, "\n"))
  }
} else {
  # Print a message if no HTML files are found
  message("No HTML files found in the /content directory.")
}

Warning message in download.file(file, destfile = file):
“URL '/content//intertopic_distance_map.html': status was 'URL using bad/illegal format or missing URL'”


ERROR: Error in download.file(file, destfile = file): cannot open URL '/content//intertopic_distance_map.html'
